In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import normalize
from scipy.sparse import hstack

from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import os
import gensim
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Getting the Data

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/DataSet.csv')
test = pd.read_csv('/content/gdrive/My Drive/DataSet_unlabel.csv')

In [0]:
train.head()

,Unnamed: 0,ID,Gene,Variation,Class,TEXT
0,0,0,FAM58A,Truncating Mutations,1,cyclin-dependent kinases cdks regulate variety...
1,1,1,CBL,W802*,2,abstract background non-small cell lung cancer...
2,2,2,CBL,Q249E,2,abstract background non-small cell lung cancer...
3,3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,4,CBL,L399V,4,oncogenic mutations monomeric casitas b-lineag...


In [0]:
test.head()

,ID,Gene,Variation,TEXT
0,0,ACSL4,R570S,2 mutation resulted myeloproliferative phenoty...
1,1,NAGLU,P521L,abstract large tumor suppressor 1 lats1 serine...
2,2,PAH,L333F,vascular endothelial growth factor receptor ve...
3,3,ING1,A148D,inflammatory myofibroblastic tumor imt neoplas...
4,4,TMEM216,G77A,abstract retinoblastoma pediatric retinal tumo...


In [0]:
#data['Com'] = data['Gene'].str.cat(data['Variation'],sep=" ").str.cat(data['TEXT'],sep=" ")
train['Gene'] = train['Gene'].apply( lambda x:x.lower() )
train['Variation'] = train['Variation'].apply( lambda x:x.lower() )
text = train[ [train.columns[2], train.columns[3], train.columns[5]] ].apply(lambda x: ' '.join(x.dropna().astype(str) ),axis=1)

In [0]:
text.head()

0    fam58a truncating mutations cyclin-dependent k...
1    cbl w802* abstract background non-small cell l...
2    cbl q249e abstract background non-small cell l...
3    cbl n454d recent evidence demonstrated acquire...
4    cbl l399v oncogenic mutations monomeric casita...
dtype: object

In [0]:
type(text)

pandas.core.series.Series

In [0]:
text = pd.DataFrame(text)

In [0]:
text.head()

,0
0,fam58a truncating mutations cyclin-dependent k...
1,cbl w802* abstract background non-small cell l...
2,cbl q249e abstract background non-small cell l...
3,cbl n454d recent evidence demonstrated acquire...
4,cbl l399v oncogenic mutations monomeric casita...


In [0]:
#data['Com'] = data['Gene'].str.cat(data['Variation'],sep=" ").str.cat(data['TEXT'],sep=" ")
test['Gene'] = test['Gene'].apply( lambda x:x.lower() )
test['Variation'] = test['Variation'].apply( lambda x:x.lower() )
text_un = pd.DataFrame( test[ [test.columns[1], test.columns[2], test.columns[3]] ].apply(lambda x: ' '.join(x.dropna().astype(str) ),axis=1))

In [0]:
text.shape

(3321, 1)

In [0]:
text_un.shape

(5668, 1)

In [0]:
text = pd.concat([ text, text_un])

In [0]:
text.shape

(8989, 1)

In [0]:
text.to_csv('/content/gdrive/My Drive/w2v_data.csv', index = False)

In [0]:
text[ text.columns[0] ].apply(lambda x: len(x.split(' '))).sum()

55146736

In [0]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [0]:
cores

2

# Training of WORD2VEC Model

In [0]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

In [0]:
def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences,
                     min_count=2,
                     window=5,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers= 4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

# Saving the trained Model
    as the data is big file model save into 3 files
    w2vmodel_big
    w2vmodel_big.trainable.syn1neg
    w2vmodel_big.wv.vectors
    

In [0]:
w2vec = get_word2vec(
    MySentences(
        text[0].values, 
        #df_test['Text'].values  Commented for Kaggle limits
    ),
    '/content/gdrive/My Drive/w2vmodel_big'
)

/content/gdrive/My Drive/w2vmodel_big not found. training model
Model done training. Saving to disk
